In [ ]:
import qutip as qt

from qubex.simulator import Coupling, System, Transmon
from qubex.simulator.optimizer_jax import PulseOptimizer
from qubex.simulator.simulator_pwc import ControlPWC, SimulatorPWC

In [ ]:
transmons = [
    Transmon(
        label="Q1",
        dimension=2,
        frequency=0.0,
        anharmonicity=-0.368,
    ),
    Transmon(
        label="Q2",
        dimension=2,
        frequency=0.01,
        anharmonicity=-0.500,
    ),
]

couplings = [
    Coupling(
        pair=("Q1", "Q2"),
        strength=0.01,
    ),
]

system = System(
    transmons=transmons,
    couplings=couplings,
)

In [ ]:
system.hamiltonian

In [ ]:
I = qt.qeye(2)
X = qt.sigmax()
target_unitary = qt.tensor(X, I)
target_unitary

In [ ]:
segment_width = 4

optimizer = PulseOptimizer(
    quantum_system=system,
    target_unitary=target_unitary,
    control_qubits=["Q1"],
    segment_count=15,
    segment_width=segment_width,
    max_rabi_frequency=0.1,
)

In [ ]:
opt_result = optimizer.optimize(
    learning_rate=0.001,
    max_iterations=1000,
    tolerance=1e-6,
)

In [ ]:
opt_result.waveforms

In [ ]:
control = ControlPWC(
    target="Q1",
    frequency=transmons[0].frequency,
    segment_values=opt_result.waveforms["Q1"],
    segment_width=segment_width,
    nsteps_per_segment=10,
)

control.plot()

In [ ]:
simulator = SimulatorPWC(system)

In [ ]:
result = simulator.simulate(
    control=control,
    initial_state="0",
)

In [ ]:
final_state = result.states[-1]
final_state

In [ ]:
final_state_target = result.substates("Q1")[-1]
final_state_target

In [ ]:
result.plot_population_dynamics()

In [ ]:
result.plot_population_dynamics("Q1")

In [ ]:
result.plot_population_dynamics("Q2")

In [ ]:
result.show_last_population("Q1")

In [ ]:
result.display_bloch_sphere("Q1")

In [ ]:
result.display_bloch_sphere("Q2")